# 2.1 Predictive Models: Mauna Loa Dataset

_In this question we will work with the Mauna Loa dataset, which contains atmospheric carbon dioxide (CO2) concentrations derived from the Scripps Institute of Oceanography’s continuous monitoring program at Mauna Loa Observatory Hawaii between 1958 and 1993._

## 2.1.1 Bayesian Linear Regression

_Write down and describe the equations involved in the Bayesian approach to linear regression, whichmay be used to describe an underlying hidden function of time given noisy data points.  You mayassume an independent Gaussian prior over each of the weights and independent noise on each datapoint._

### Solution

As used in lecture 2, we can easily write down the mathematical form the of the Bayesian linear regression

\begin{equation}
    y = \omega X^{\top} + \epsilon
\end{equation}
\begin{equation}
    \text{e.g. } \epsilon \sim \mathcal{N}(\mu, \sigma^{2})
\end{equation}

The likelihood function then follows the following form according to Bishop

\begin{equation}
    p(y | X, \omega) = \mathcal{N}(\omega X_{\top}, \frac{1}{\sigma^{2}}).
\end{equation}

The prior distribution for the likelihood is given by

\begin{equation}
    p(\omega_{0}) = \mathcal{N}(m_{0}, S_{0})
\end{equation}

and its posterior distribution is given by

\begin{equation}
    p(\omega_{i+1} | \omega_{i}, x_{i}, y_{i}) = \mathcal{N}(m_{i+1}, S_{i+1})
\end{equation}
\begin{equation}
    S_{i+1} = \left(S^{-1}_{i} + \beta x^{\top}_{i} x_{i} \right)^{-1}
\end{equation}
\begin{equation}
    m_{i+1} = S_{i+1} \left( S^{-1}_{i} m_{i} + \beta x_{i} y_{i} \right)
\end{equation}

## 2.2.2 Connection between Bayesian Linear Regression and Gaussian Processes

_By considering the covariance of the function in (2.1) at two time points, show how we can derive an equivalent kernel function using an inner product of basis functions. Hence show that the Bayesian linear regression approach is equivalent to the Gaussian process approach with a suitable choice of kernel function._

### Solution

To begin, note that the following two notations of Bayesian linear regression and Gaussian processes are equivalent

\begin{equation}
    y = \omega^{\top}X, \quad \omega \sim \mathcal{N}(0, 1_{\{ d \times d \}}) \Longleftrightarrow f \sim \mathcal{GP}(0, K), \quad K(x, x') = x^{\top} x'
\end{equation}

where the kernel of the Gaussian process is the linear kernel. So let's assume we do multiple draws from the Bayesian Linear Regression, then we have multiple function values, i.e.

\begin{equation}
    \{ y_{i} = X_{i} \omega \}_{i \in \mathcal{S}}
\end{equation}

where thence all points have to lie in the same plane, as determined by $\omega$. If we now repeat the same procedure with the Gaussian process, we obtain a set

\begin{equation}
    \{ y_{i} \sim \mathcal{N}(0, X_{i}X_{i}^{\top}) \}_{i \in \mathcal{S}'}.
\end{equation}

If we now diagonalize covariance Kernel, then we obtain

\begin{equation}
    X_{i}X_{i}^{\top} = V \Lambda V^{\top}
\end{equation}

where only the first d diagonal entries are non-zero, armed with which we can then rewrite the set of Gaussian process realizations as

\begin{equation}
    \{ y_{i} = V \Lambda^{\frac{1}{2}} z \}_{i \in \mathcal{S}'}.
\end{equation}

Due to the invariance of the Gaussian distribution under linear transformations, see tutorial 1, we can hence conclude that z can be constrained to be a d-dimensional random variable $\tilde{z}$

\begin{equation}
    f_{i} \overset{d}{=} X_{i} \tilde{z}, \quad \tilde{z} \sim \mathcal{N}(0, 1_{\{ d \times d \}}).
\end{equation}

Hence showing the equivalence of the algorithms, when considering Gaussian processes with linear kernels.

## 2.2.3 Gaussian Processes

_Using a Gaussian process, construct a predictive model of CO2 emissions over the subsequent 20 years. Give full details of any assumptions and equations you make use of when fitting your model to the data._

### Solution

In [1]:
import os
import torch
import tqdm
import gpytorch
import pandas as pd
import torch.nn as nn
from gpytorch.means import ConstantMean, LinearMean
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.variational import VariationalStrategy, CholeskyVariationalDistribution
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ApproximateGP, GP
from gpytorch.mlls import VariationalELBO, AddedLossTerm
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.models.deep_gps import DeepGPLayer, DeepGP
from gpytorch.mlls import DeepApproximateMLL

We begin by loading the data into our notebook and instantiating it as Tensors

In [2]:
dataset = pd.read_csv("../Coursework/CO2_Mauna_Loa_Data.csv")

In [3]:
dataset.columns

Index(['months since 1960-01-01', 'co2 ppmv'], dtype='object')

In [4]:
X = dataset['months since 1960-01-01']
y = dataset['co2 ppmv']

X_tensor = torch.tensor(X.to_numpy(), dtype=torch.float32)
y_tensor = torch.tensor(y.to_numpy(), dtype=torch.float32)

X_tensor = X_tensor.view((len(X_tensor), 1))
y_tensor = y_tensor.view((len(y_tensor), 1))


train_x, train_y = X_tensor, y_tensor

In [5]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)

After which we can now begin to define our model. In this case here we are using a deep Gaussian process for which we have to first define its first layer

In [6]:
class DeepGPHiddenLayer(DeepGPLayer):

    def __init__(self, input_dims, output_dims, num_inducing=128, mean_type='constant'):
        if output_dims is None:
            inducing_points = torch.randn(num_inducing, input_dims)
            batch_shape = torch.Size([])
        else:
            inducing_points = torch.randn(output_dims, num_inducing, input_dims)
            batch_shape = torch.Size([output_dims])

        # Define the variational distribution
        variational_distribution = CholeskyVariationalDistribution(
            num_inducing_points=num_inducing,
            batch_shape=batch_shape
        )

        # Define the variational strategy
        variational_strategy = VariationalStrategy(
            self,
            inducing_points,
            variational_distribution,
            learn_inducing_locations=True
        )

        super(DeepGPHiddenLayer, self).__init__(variational_strategy, input_dims, output_dims)

        self.mean_module = ConstantMean(batch_shape=batch_shape)
        self.covar_module = ScaleKernel(
            RBFKernel(batch_shape=batch_shape, ard_num_dims=input_dims),
            batch_shape=batch_shape, ard_num_dims=None
        )
        self.linear_layer = nn.Linear(input_dims, 1)
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

After which we can then construct the actual deep Gaussian process. Here the forward pass then passes through the hidden layers, which could in theory be many more than the ones used here. But this choice boils down to the amount of data we have available

In [7]:
num_output_dims = 1

class DeepGP(DeepGP):

    def __init__(self, train_x_shape):
        hidden_layer = DeepGPHiddenLayer(
            input_dims=train_x_shape[-1],
            output_dims=num_output_dims,
            mean_type='linear',
        )
        last_layer = DeepGPHiddenLayer(
            input_dims=hidden_layer.output_dims,
            output_dims=None,
            mean_type='constant'
        )

        super().__init__()

        self.hidden_layer = hidden_layer
        self.last_layer = last_layer
        self.likelihood = GaussianLikelihood()
    
    def forward(self, inputs):
        hidden_rep1 = self.hidden_layer(inputs)
        return self.last_layer(hidden_rep1)
    
    def predict(self, test_loader):
        with torch.no_grad():
            mus = []
            variances = []
            lls = []
            for x_batch, y_batch in test_loader:
                preds = model.likelihood(model(x_batch))
                mus.append(preds.mean)
                variances.append(preds.variance)
                lls.append(model.likelihood.log_marginal(y_batch, model(x_batch)))
        
        return torch.cat(mus, dim=-1), torch.cat(variances, dim=-1), torch.cat(lls, dim=-1)

In [8]:
model = DeepGP(train_x.shape)

With which we can now move to the definition of the training loop

In [9]:
num_epochs = 10
num_samples = 50

Define the optimizer and marginal log-likelihood

In [10]:
optimizer = torch.optim.Adam([{'params': model.parameters()},], lr=0.01)
mll = DeepApproximateMLL(VariationalELBO(model.likelihood, model, train_x.shape[-2]))

Where we can now define the actual training loop

In [11]:
epochs_iter = tqdm.tqdm_notebook(range(num_epochs), desc="Epoch")
for i in epochs_iter:
    minibatch_iter = tqdm.tqdm_notebook(train_loader, desc="Minibatch", leave=False)
    for x_batch, y_batch in minibatch_iter:
        with gpytorch.settings.num_likelihood_samples(num_samples):
            optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            loss.backward()
            optimizer.step()

            minibatch_iter.set_postfix(loss=loss.item())

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/449 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/449 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/449 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/449 [00:00<?, ?it/s]

...

Minibatch:   0%|          | 0/449 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/449 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/449 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/449 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/449 [00:00<?, ?it/s]

With which we can then assess our prediction for the next 20 months

In [14]:
x_predict_batch = torch.arange(431.5, 451)
x_predict_batch = x_predict_batch.view((len(x_predict_batch), 1))

In [16]:
with torch.no_grad():
    mus = []
    variances = []
    preds = model.likelihood(model(x_predict_batch))
    mus.append(preds.mean)
    variances.append(preds.variance)